In [ ]:
 !pip install -q tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00


In [ ]:
import math
import os
import time
from dataclasses import dataclass
import inspect    #helps with inspecting properties within an object
import torch
import torch.nn as nn
from torch.nn import functional as F

#evaluation
from hellaswag import render_example, iterate_examples

In [ ]:
# device agnostic code (DDP) n_number of GPUs
# run command: "torchrun --standalone --nproc_per_node=n_number <filename>"
from torch.distributed import init_process_group, destroy_process_group
from torch.nn.parrallel import DistributedDataParallel as DDP
import torch.distributed as dist

#RANK: gpu assignment
#WORLD_SIZE = number of GPUs
#LOCAL_RANK = gpu rank on box, for multi-gpu box

ddp = int(os.environ.get("RANK", -1))  != -1  #gets the gpu count else -1

if ddp:
  assert torch.cuda.is_available() # CUDA is required since this is a GPU. ddp can be done on CPU too
  init_process_group(backend='nccl')  #Nvidia collective compute library: for multi-gpu operations
  ddp_rank = int(os.environ['RANK'])
  ddp_world_size = int(os.environ['WORLD_SIZE'])
  ddp_local_rank = int(os.environ['LOCAL_RANK'])
  device = f'cuda:{ddp_local_rank}'
  torch.cuda.set_device(device)
  master_process = ddp_rank == 0   # == has a higher precedence.

else:
  #without ddp
  ddp_rank = 0
  ddp_local_rank = 0
  ddp_world_size = 1   #assuming there is one gpu
  master_process = True

  #device agnostic code
  device = 'cpu'
  if torch.cuda.is_available():
    device = 'cuda'
  elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = 'mps'

  print(f"{device=}")



Using device: cuda


In [ ]:
class CausalSelfAttention(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        # regularization
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        # not really a 'bias', more of a mask, but following the OpenAI/HF naming though
        self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                                     .view(1, 1, config.block_size, config.block_size))
        self.c_proj.NANOGPT_WEIGHT_INIT = 1.0

    def forward(self, x):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)
        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        # nh is "number of heads", hs is "head size", and C (number of channels) = nh * hs
        # e.g. in GPT-2 (124M), n_head=12, hs=64, so nh*hs=C=768 channels in the Transformer
        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

        # # Normal attention (materializes the large (T,T) matrix for all the queries and keys)
        # att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        # att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
        # att = F.softmax(att, dim=-1)
        # y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)

        """Switching to Flash Attention"""
        y = F.scaled_dot_product_attention(q, k, v, is_causal=True)
        # continuation same for causalattention
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side
        # output projection
        y = self.c_proj(y)
        return y

class MLP(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.c_fc    = nn.Linear(config.n_embd, 4 * config.n_embd)
        self.gelu    = nn.GELU(approximate='tanh')
        self.c_proj  = nn.Linear(4 * config.n_embd, config.n_embd)
        self.NANOGPT_WEIGHT_INIT = 1.0     #associated with the projection

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x

class Block(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x

@dataclass
class GPTConfig:
    block_size: int = 1024 # max sequence length
    vocab_size: int = 50257 # number of tokens: 50,000 BPE merges + 256 bytes tokens + 1 <|endoftext|> token
    n_layer: int = 12 # number of layers
    n_head: int = 12 # number of heads
    n_embd: int = 768 # embedding dimension

class GPT(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = nn.LayerNorm(config.n_embd),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        self.transformer.wte.weight = self.lm_head.weight  #weight sharing scheme linear.weight.shape(vocab_size, n_embd)

        # apply this at initialization. we could have set this while defining each layer
        self.apply(self._init_weights) #applies a function to an nn.Module object

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
          sd = 0.02
          if hasattr(module, "NANOGPT_WEIGHT_INIT"):
            sd *= (2 * self.config.n_layer)**-0.5
          torch.nn.init.normal_(module.weight, mean=0.0, std=sd)
          if module.bias is not None:
            torch.nn.init.zeros_(module.bias)         #initializing bias. there is 'torch.nn.init.ones_'
        elif isinstance(module, nn.Embedding):
          torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)   #intializing weights

    def forward(self, idx, targets=None):
      #input shape is B, T
      B, T = idx.size()
      assert T <= self.config.block_size, f"cannot forward sequence of length {T} because the block size is only {self.config.block_size}"
      #position and token embedding
      pos = torch.arange(0, T, dtype=torch.long, device=idx.device) #input would already have been sent to device
      pos_emb = self.transformer.wpe(pos) # (T, n_embd)  #module dict can be accessed using '.'
      tok_emb = self.transformer.wte(idx) # (B, T, n_embd)
      x = tok_emb + pos_emb
      #blocks
      for block in self.transformer.h:
        x = block(x)
      #LayerNorm
      x = self.transformer.ln_f(x)

      logits = self.lm_head(x)  #B, T, vocab_size

      loss = None
      if targets is not None:
        loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))  # logits should be 2d, targets should be 1d  (B*T, vocab_size, B*T)
      return logits, loss



    @classmethod
    def from_pretrained(cls, model_type):
        """Loads pretrained GPT-2 model weights from huggingface"""
        assert model_type in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
        from transformers import GPT2LMHeadModel
        print("loading weights from pretrained gpt: %s" % model_type)

        # n_layer, n_head and n_embd are determined from model_type
        config_args = {
            'gpt2':         dict(n_layer=12, n_head=12, n_embd=768),  # 124M params
            'gpt2-medium':  dict(n_layer=24, n_head=16, n_embd=1024), # 350M params
            'gpt2-large':   dict(n_layer=36, n_head=20, n_embd=1280), # 774M params
            'gpt2-xl':      dict(n_layer=48, n_head=25, n_embd=1600), # 1558M params
        }[model_type]
        config_args['vocab_size'] = 50257 # always 50257 for GPT model checkpoints
        config_args['block_size'] = 1024 # always 1024 for GPT model checkpoints
        # create a from-scratch initialized minGPT model
        config = GPTConfig(**config_args)
        model = GPT(config)
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')] # discard this mask / buffer, not a param

        # init a huggingface/transformers model
        model_hf = GPT2LMHeadModel.from_pretrained(model_type)
        sd_hf = model_hf.state_dict()

        # copy while ensuring all of the parameters are aligned and match in names and shapes
        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')] # ignore these, just a buffer
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')] # same, just the mask (buffer)
        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
        # basically the openai checkpoints use a "Conv1D" module, but we only want to use a vanilla Linear
        # this means that we have to transpose these weights when we import them
        assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):
                # special treatment for the Conv1D weights we need to transpose
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                # vanilla copy over the other parameters
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])

        return model

    #configure optimizer
    def configure_optimizers(self, weight_decay, learning_rate, device):
        # start with all of the candidate parameters (that require grad)
        param_dict = {pn: p for pn, p in self.named_parameters()}
        param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}
        # create optim groups. Any parameters that is 2D will be weight decayed, otherwise no.
        # i.e. all weight tensors in matmuls + embeddings decay, all biases and layernorms don't.
        decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
        nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
        optim_groups = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': nodecay_params, 'weight_decay': 0.0}
        ]
        num_decay_params = sum(p.numel() for p in decay_params)
        num_nodecay_params = sum(p.numel() for p in nodecay_params)
        if master_process:
            print(f"num decayed parameter tensors: {len(decay_params)}, with {num_decay_params:,} parameters")
            print(f"num non-decayed parameter tensors: {len(nodecay_params)}, with {num_nodecay_params:,} parameters")
        # Create AdamW optimizer and use the fused version if it is available
        fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
        use_fused = fused_available and 'cuda' in device
        if master_process:
           print(f"using fused AdamW: {use_fused}")
        optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=(0.9, 0.95), eps=1e-8, fused=use_fused)
        return optimizer

In [ ]:
import tiktoken
import numpy as np

def load_tokens(filename):
  npt = np.load(filename)
  npt = npt.astype(np.int32)  #not really required
  ptt = torch.tensor(npt, dtype=torch.long)
  return ptt


"""creating an iterative dataloader class"""
class DataLoaderLite:
    def __init__(self, B, T, process_rank, num_processes, split):  #process_rank ensures same batch is not used by different gpus
        self.B = B
        self.T = T
        self.process_rank = process_rank
        self.num_processes = num_processes

        # load fineweb data
        assert split in {"train", "val"}

        # get shard
        data_root = "edu_fineweb10B"
        shards = os.listdir(data_root)
        shards = [s for s in shards if split in s]
        shards = sorted(shards)
        shards = [os.path.join(data_root, s) for s in shards]
        self.shards = shards

        assert len(shards) > 0, f"No data shard found for {split}"
        if master_process:
            print(f"loaded {len(self.tokens)} tokens")
            print(f"found {len(shards)} shards for split {split}")
            # print(f"1 epoch = {len(self.tokens) // ((B * T) / ddp_world_size)} batches")  #with shakespare.txt

        self.reset()     #reset for new shard

        #fineweb
    def reset(self):
        self.current_shard = 0
        self.tokens = load_tokens(self.shards[self.current_shard])
        self.current_position = B * T * process_rank

    def next_batch(self):
        B, T = self.B, self.T
        buf = self.tokens[self.current_position : self.current_position+B*T+1]
        x = (buf[:-1]).view(B, T) # inputs
        y = (buf[1:]).view(B, T) # targets
        # advance the position in the tensor
        # self.curr_pos += (B*T)+(B*T*torch.randint(-3, 3, (1,)).item()) #randomly assigns batches. add condition for negative positions.
        self.current_position += B * T * self.num_processes
        # if loading the next batch would be out of bounds, reset
        if self.current_position + (B * T * self.num_processes + 1) > len(self.tokens):
            # self.current_position = B * T * self.process_rank  #shakespare.txt
            self.current_shard = (self.current_shard + 1) % len(self.shards)
            self.tokens = load_tokens(self.shards[self.current_shard])
            self.current_position = B * T * self.process_rank
        return x, y


# helper function for HellaSwag eval
# takes tokens, mask, and logits, returns the index of the completion with the lowest loss

def get_most_likely_row(tokens, mask, logits):
    # evaluate the autoregressive loss at all positions
    shift_logits = (logits[..., :-1, :]).contiguous()
    shift_tokens = (tokens[..., 1:]).contiguous()
    flat_shift_logits = shift_logits.view(-1, shift_logits.size(-1))
    flat_shift_tokens = shift_tokens.view(-1)
    shift_losses = F.cross_entropy(flat_shift_logits, flat_shift_tokens, reduction='none')
    shift_losses = shift_losses.view(tokens.size(0), -1)
    # now get the average loss just for the completion region (where mask == 1), in each row
    shift_mask = (mask[..., 1:]).contiguous() # we must shift mask, so we start at the last prompt token
    masked_shift_losses = shift_losses * shift_mask
    # sum and divide by the number of 1s in the mask
    sum_loss = masked_shift_losses.sum(dim=1)
    avg_loss = sum_loss / shift_mask.sum(dim=1)
    # now we have a loss for each of the 4 completions
    # the one with the lowest loss should be the most likely
    pred_norm = avg_loss.argmin().item()
    return pred_norm


In [ ]:

torch.manual_seed(42)
if torch.cuda.is_available():
  torch.cuda.manual_seed(42)

enc = tiktoken.get_encoding('gpt2')
#TF32 matmuls
torch.set_float32_matmul_precision('high')  # TF32 computation, 'medium' is bf16

# #without gradient accumulation
# train_data = DataLoaderLite(16, 1024)

#with gradient accumulation
total_batch_size = 524288    #A nice number, close to the OpenAI batch size
B = 4
T = 1024
assert total_batch_size % (B*T) == 0  #confirm divisibility for grad accumulation
grad_accum_steps = total_batch_size // (B * T * ddp_world_size)
if master_process:  #all prints if not we'll have no_gpu prints
    print(f"desired batch size according to OpenAI: {total_batch_size}, grad accumulation steps: {grad_accum_steps}")
assert total_batch_size % (B * T * ddp_world_size) == 0, "for grad accumulation to be possible at full gpu performance"
train_loader = DataLoaderLite(B, T, ddp_rank, ddp_world_size, split="train")
val_loader = DataLoaderLite(B, T, ddp_rank, ddp_world_size, split="val")

# model = GPT(GPTConfig())
model = GPT(GPTConfig(vocab_size=50304))  #increasing the trainable vocab_size, using nice number for even computation
model.to(device)

#add torch.compile
use_compile = False
if use_compile:
    model = torch.compile(model)

# ddp model configuration
if ddp:
  model = DDP(model, device_ids=[ddp_local_rank])
raw_model = model.module if ddp else model     #for optimizer only

#creating a log dir to write logs and checkpoints to
log_dir = "log"
os.makedirs(log_dir, exist_ok=True)
log_file = os.path.join(log_dir, "log.txt")
with open(log_file, "w") as f:  #open for writing
  pass
#adding a learning rate scheduler (OpenAI uses a cosine lr schedule)
max_lr = 6e-4
min_lr = max_lr * 0.1
warmup_steps = 715      #as per OpenAI's settings
max_steps = 19073       #as per OpenAI's settings

def get_lr(it):
  if it < warmup_steps:
    return max_lr * (it+1) /warmup_steps
  if it > max_steps:
    return min_lr

  decay_ratio = (it - warmup_steps) / (max_steps - warmup_steps)
  assert 0 <= decay_ratio <= 1
  coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio))  # start at one and decay to zero

  return min_lr + coeff * (max_lr - min_lr)


# optimizer
# optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)
# optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, betas=(0.9, 0.95), eps=1e-8)    #OpenAI settings
optimizer = raw_model.configure_optimizers(weight_decay=0.1, learning_rate=6e-4, device=device)

#training loop
for step in range(max_steps):
  t0 = time.time()
  last_step = (step == max_steps - 1)

  # validation loss code, once in a while
  if step % 250 == 0 or last_step:
    model.eval()
    val_loader.reset()
    with torch.no_grad():
      val_loss_accum = 0.0
      val_loss_steps = 20
      for _ in range(val_loss_steps):
        x, y = val_loader.next_batch()
        x, y = x.to(device), y.to(device)
        with torch.autocast(device_type=device, dtype=torch.bfloat16):
          logits, loss = model(x, y)
          loss = loss / val_loss_steps
          val_loss_accum += loss.detach()
    if ddp:
      dist.all_reduce(val_loss_accum, op=dist.ReduceOp.AVG)
    if master_process:
      print(f"validation loss: {val_loss_accum.item():.4f}")
      with open(log_file, "a") as f:
                f.write(f"{step} val {val_loss_accum.item():.4f}\n")

      #occationally saving checkpoints
      checkpoint_path = os.path.join(log_dir, f'model_{step:5d}.pt')
      checkpoint = {
          "model": raw_model.state_dict(),
          "config": raw_model.config,
          "step": step,
          "val_loss": val_loss_accum.item(),
          "optimizer": optimizer.state_dict(),
          "rng_state": torch.get_rng_state(),
          "cuda_rng_state": torch.cuda.get_rng_state_all() if torch.cuda.is_available() else None,
      }   #rng is used for resuming training on gpu
      torch.save(checkpoint, checkpoint_path)    #checkpoint file is a dict. there a lot other stuffs you'd want to add


  # once in a while evaluate hellaswag written to file
  if (step % 250 == 0 or last_step) and (not use_compile):
      num_correct_norm = 0
      num_total = 0
      for i, example in enumerate(iterate_examples("val")):
          # only process examples where i % ddp_world_size == ddp_rank
          if i % ddp_world_size != ddp_rank:
              continue
          # render the example into tokens and labels
          _, tokens, mask, label = render_example(example)
          tokens = tokens.to(device)
          mask = mask.to(device)
          # get the logits
          with torch.no_grad():
              with torch.autocast(device_type=device, dtype=torch.bfloat16):
                  logits, loss = model(tokens)
              pred_norm = get_most_likely_row(tokens, mask, logits)
          num_total += 1
          num_correct_norm += int(pred_norm == label)
      # reduce the stats across all processes
      if ddp:
          num_total = torch.tensor(num_total, dtype=torch.long, device=device)
          num_correct_norm = torch.tensor(num_correct_norm, dtype=torch.long, device=device)
          dist.all_reduce(num_total, op=dist.ReduceOp.SUM)
          dist.all_reduce(num_correct_norm, op=dist.ReduceOp.SUM)
          num_total = num_total.item()
          num_correct_norm = num_correct_norm.item()
      acc_norm = num_correct_norm / num_total
      if master_process:
          print(f"HellaSwag accuracy: {num_correct_norm}/{num_total}={acc_norm:.4f}")
          with open(log_file, "a") as f:
              f.write(f"{step} hella {acc_norm:.4f}\n")

  #Timely Evaluation on terminal
  #evaluating the model once in a while
  # for now, we'll have to disable torch compile for this to work well.
  if ((step > 0 and step % 250 == 0) or last_step) and (use_compile):
      model.eval()
      num_return_sequences=4
      max_length = 32
      tokens = enc.encode("Hello, I'm a language model") # (8,)
      tokens = torch.tensor(tokens, dtype=torch.long)
      tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1)  #(5, 8)  B, T
      xgen = tokens.to(device)
      sample_rng = torch.Generator(device=device)  #deterministic results
      sample_rng.manual_seed(45 + ddp_rank)

      while xgen.size(1) < max_length:
        # forward pass
        with torch.no_grad():
          with torch.autocast(device_type=device, dtype=torch.bfloat16):
              logits, loss = model(xgen)  #B, T, vocab_size
          #take the logits at last hidden position
          logits = logits[:, -1, :]    #B, vocab_size

          probs = F.softmax(logits, dim=1)  #we always do softmax on the last dim

          # top-K  changes from (5, vocab_size) to (5, 50)
          topk_probs, top_indices = torch.topk(probs, 50, dim=-1)

          ix = torch.multinomial(topk_probs, 1, generator=sample_rng)  #B, 1
          #because our topk_prob is a sample, we'll have to retrieve its indicies
          xcol = torch.gather(top_indices, -1, ix)   #B, 1    index ix from top_indices and gather it in the column
          #append to sequence
          x = torch.cat((xgen, xcol), dim=1)


      # print generated text
      for i in range(num_return_sequences):
        tokens = xgen[i, :max_length].tolist()    #can't use decode on tensor
        decoded = enc.decode(tokens)
        print(f"ddp rank {ddp_rank} sample {i}: {decoded}")

  model.train()    #set back in training mode
  optimizer.zero_grad()  #never forget to zero the gradient after each step

  #with gradient accumulation
  loss_accum = 0.0
  for micro_step in range(grad_accum_steps):
    x, y = train_loader.next_batch()
    x, y = x.to(device), y.to(device)

    #no-sync should also affect the forward pass
    if ddp:
      model.require_backward_grad_sync = (micro_step == grad_accum_steps - 1)

    #cast loss and logits operation to bfloat16
    with torch.autocast(device_type=device, dtype=torch.bfloat16):
      logits, loss = model(x, y)  #a lot aof changes can be made


    loss = loss / grad_accum_steps  #this is relevant otherwise the actual value would equal our value divided by accum_no. check with small example
    loss_accum += loss.detach()  #detach instead of item because detach preserves value as tensor (its possible to += a tensor to a scalar value)
    loss.backward()
    if ddp:
      dist.all_reduce(loss_accum, op=dist.ReduceOp.AVG)

  # clip gradient to not exceed 1
  norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

  #include lr from scheduler
  lr = get_lr(step)
  for param_group in optimizer.param_groups:  #optimizer object has an attribute called param_groups
    param_group['lr'] = lr

  optimizer.step()
  torch.cuda.synchronize()  #synchronize the cpu and gpu times.
  t1 = time.time()
  dt = t1 - t0

  # #without grad accum
  # tokens_processed = train_loader.B * train_loader.T

  #with grad accum
  tokens_processed = grad_accum_steps * train_loader.B * train_loader.T

  tokens_per_sec = tokens_processed / dt
  if master_process:
        print(f"step{step:4d} | loss: {loss_accum.item():.6f} | {lr:.4e} | norm: {norm} | dt:{dt*1000:.2f}ms | tok/sec: {tokens_per_sec}")
        with open(log_file, "a") as f:               #a is the mode for add, same as c
          f.write(f"{step} train {loss_accum.item():.6f}\n")